In [1]:
import h5py    
import numpy as np
import pandas as pd
import datetime
from termcolor import colored

### Take dataset with hdf5 format

In [2]:
sets = ['dev', 'test']
filename = 'N-CMAPSS_DS03-012.h5'

### Read file and rename columns

In [3]:
with h5py.File(filename, 'r') as hdf:
        
    for s in sets:

        globals()[f"theta_{s}"] = pd.DataFrame(np.array(hdf.get(f'T_{s}')), 
                                                       columns=np.array(hdf.get('T_var')).astype('U'))
        globals()[f"xv_{s}"] = pd.DataFrame(np.array(hdf.get(f'X_v_{s}')), 
                                                   columns=np.array(hdf.get('X_v_var')).astype('U'))
        globals()[f"xs_{s}"] = pd.DataFrame(np.array(hdf.get(f'X_s_{s}')), 
                                                   columns=np.array(hdf.get('X_s_var')).astype('U'))
        globals()[f"w_{s}"] = pd.DataFrame(np.array(hdf.get(f'W_{s}')), 
                                                   columns=np.array(hdf.get('W_var')).astype('U'))
        globals()[f"aux_{s}"] = pd.DataFrame(np.array(hdf.get(f'A_{s}')), 
                                                   columns=np.array(hdf.get('A_var')).astype('U')).astype('int64')

        globals()[f"y_{s}"] = pd.DataFrame(np.array(hdf.get(f'Y_{s}')), columns=['RUL'])


        #### RENAME COLUMNS

        globals()[f"w_{s}"].rename(columns={'alt':'Altitude'}, inplace=True)

        globals()[f"aux_{s}"].rename(columns={'unit':'Unit','cycle':'Cycle','Fc':'Class','hs':'State'},
                                           inplace=True)
        globals()[f"theta_{s}"].rename(columns={'fan_eff_mod':'Fan_eff_mod','fan_flow_mod':'Fan_flow_mod'},
                                             inplace=True)
            
    

### Check if there are observations with missing values

In [4]:
dev_tmp = pd.concat([aux_dev, w_dev, xs_dev, xv_dev, theta_dev, y_dev], axis=1)


print(f'The dev dataset comprises of {len(dev_tmp):,} observations and there are {len(dev_tmp) - len(dev_tmp.dropna())} missing values\n\n'
      f'The features include: {len(theta_dev.columns)} '.replace(',','.') +
      f'health parameters, {len(xs_dev.columns)} sensors, {len(xv_dev.columns)} virtual sensors, '
      f'{len(aux_dev.columns)} auxiliary data, {len(w_dev.columns)} scenario descriptors and {len(y_dev.columns)} '
      f'output variable\n')
    
del(dev_tmp)
    

test_tmp = pd.concat([aux_test, w_test, xs_test, xv_test, theta_test, y_test], axis=1)

print(f'The test dataset comprises of {len(test_tmp):,} observations and there are {len(test_tmp) - len(test_tmp.dropna())} missing values\n\n'
      f'The features include: {len(theta_test.columns)} '.replace(',','.') + 
      f'health parameters, {len(xs_test.columns)} sensors, {len(xv_test.columns)} virtual sensors, '
      f'{len(aux_test.columns)} auxiliary data, {len(w_test.columns)} scenario descriptors and {len(y_test.columns)} '
      f'output variable')
    
del(test_tmp)

The dev dataset comprises of 5.571.277 observations and there are 0 missing values

The features include: 10 health parameters, 14 sensors, 14 virtual sensors, 4 auxiliary data, 4 scenario descriptors and 1 output variable

The test dataset comprises of 4.251.560 observations and there are 0 missing values

The features include: 10 health parameters, 14 sensors, 14 virtual sensors, 4 auxiliary data, 4 scenario descriptors and 1 output variable


## Quick analysis of the two datasets

In [5]:
dev = pd.concat([aux_dev, y_dev], axis=1)
test = pd.concat([aux_test, y_test], axis=1)

### How many units and fight classes

The units are divided into three flight classes depending on whether the unit is operating short-length flights (i.e., flight class 1), medium-length flights (i.e., flight class 2), or long-length flights (i.e., flight class 3)

| Flight Class   | Flight Length [h]
| :-----------:  | :-----------:    
| 1              |    1 to 3        
| 2              |    3 to 5        
| 3              |    5 to 7        

In [6]:
print("The dev units: ", end = "")
print(*aux_dev['Unit'].unique(), sep = ", ", end = "  and  ")
print("the test units: ", end = "")
print(*aux_test['Unit'].unique(), sep = ", ")  


print("\nThe dev flight classes: ", end = "") 
print(*np.sort(aux_dev['Class'].unique()), sep = ", ", end = "  and  ")
print("the test flight classes: ", end = "") 
print(*np.sort(aux_test['Class'].unique()), sep = ", ")

The dev units: 1, 2, 3, 4, 5, 6, 7, 8, 9  and  the test units: 10, 11, 12, 13, 14, 15

The dev flight classes: 1, 2, 3  and  the test flight classes: 1, 2, 3


The dev units are different from the test ones. Each unit has its proper flight class

In [7]:
unit_data = {}

for unit in list(dev['Unit'].unique()):
    
    unit_data[unit] = dev[(dev['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)
    
    print(f"Dev unit {unit} has flight class: {unit_data[unit]['Class'].unique()[0]}")
    
for unit in list(test['Unit'].unique()):
    
    unit_data[unit] = test[(test['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)
    
    print(f"Test unit {unit} has flight class: {unit_data[unit]['Class'].unique()[0]}")

Dev unit 1 has flight class: 1
Dev unit 2 has flight class: 2
Dev unit 3 has flight class: 2
Dev unit 4 has flight class: 2
Dev unit 5 has flight class: 1
Dev unit 6 has flight class: 3
Dev unit 7 has flight class: 2
Dev unit 8 has flight class: 3
Dev unit 9 has flight class: 1
Test unit 10 has flight class: 3
Test unit 11 has flight class: 3
Test unit 12 has flight class: 1
Test unit 13 has flight class: 3
Test unit 14 has flight class: 1
Test unit 15 has flight class: 2


### Check if cycle and rul are perfectly linear

In [8]:
for unit in list(dev['Unit'].unique()):
    
    cycle = 1
    rul = unit_data[unit]["RUL"].iloc[0]
    
    while rul >= 0:
    
        if not (unit_data[unit][unit_data[unit]['Cycle'] == cycle].equals(unit_data[unit][unit_data[unit]['RUL'] == rul])):
            print('Not linear')
        
        cycle = cycle + 1
        rul = rul - 1
    
    
for unit in list(test['Unit'].unique()):
    
    cycle = 1
    rul = unit_data[unit]["RUL"].iloc[0]
    
    while rul >= 0:
    
        if not (unit_data[unit][unit_data[unit]['Cycle'] == cycle].equals(unit_data[unit][unit_data[unit]['RUL'] == rul])):
            print('Not linear')
        
        cycle = cycle + 1
        rul = rul - 1


### Find the transition times
The time in which the transition from normal to abnormal degradation starts, i.e. the onset of a fault, is called transition time t<sub>s</sub>.

In [9]:
ts = {}
for unit in list(dev['Unit'].unique()):

    ts[unit] = unit_data[unit][unit_data[unit]['State'] == 0]["Cycle"].iloc[0]
    print(f'Dev unit {unit}: {unit_data[unit]["Cycle"].iloc[-1]  - ts[unit]} remaining cycles')

for unit in list(test['Unit'].unique()):

    ts[unit] = unit_data[unit][unit_data[unit]['State'] == 0]["Cycle"].iloc[0]
    print(f'Test unit {unit}: {unit_data[unit]["Cycle"].iloc[-1]  - ts[unit]} remaining cycles')

Dev unit 1: 35 remaining cycles
Dev unit 2: 49 remaining cycles
Dev unit 3: 42 remaining cycles
Dev unit 4: 36 remaining cycles
Dev unit 5: 55 remaining cycles
Dev unit 6: 45 remaining cycles
Dev unit 7: 55 remaining cycles
Dev unit 8: 53 remaining cycles
Dev unit 9: 47 remaining cycles
Test unit 10: 48 remaining cycles
Test unit 11: 40 remaining cycles
Test unit 12: 56 remaining cycles
Test unit 13: 58 remaining cycles
Test unit 14: 40 remaining cycles
Test unit 15: 42 remaining cycles


In [10]:
ts = {}
for unit in list(dev['Unit'].unique()):

    ts[unit] = unit_data[unit][unit_data[unit]['State'] == 0]["Cycle"].iloc[0]
    print(f'Transition time of dev unit {unit}: {ts[unit]} / {unit_data[unit]["Cycle"].iloc[-1]} total cycles')

print('\n')
for unit in list(test['Unit'].unique()):

    ts[unit] = unit_data[unit][unit_data[unit]['State'] == 0]["Cycle"].iloc[0]
    print(f'Transition time of test unit {unit}: {ts[unit]} / {unit_data[unit]["Cycle"].iloc[-1]} total cycles')

Transition time of dev unit 1: 37 / 72 total cycles
Transition time of dev unit 2: 24 / 73 total cycles
Transition time of dev unit 3: 25 / 67 total cycles
Transition time of dev unit 4: 24 / 60 total cycles
Transition time of dev unit 5: 38 / 93 total cycles
Transition time of dev unit 6: 18 / 63 total cycles
Transition time of dev unit 7: 25 / 80 total cycles
Transition time of dev unit 8: 18 / 71 total cycles
Transition time of dev unit 9: 37 / 84 total cycles


Transition time of test unit 10: 18 / 66 total cycles
Transition time of test unit 11: 19 / 59 total cycles
Transition time of test unit 12: 37 / 93 total cycles
Transition time of test unit 13: 19 / 77 total cycles
Transition time of test unit 14: 36 / 76 total cycles
Transition time of test unit 15: 25 / 67 total cycles


### Check if any fault data occur in time t<sub>s</sub>

In [11]:
for unit in list(dev['Unit'].unique()):
    
    if not (unit_data[unit][unit_data[unit]['State'] == 0].equals(unit_data[unit][unit_data[unit]['Cycle'] >= ts[unit]])):
        print('Not corresponding')

for unit in list(test['Unit'].unique()):
    
    if not (unit_data[unit][unit_data[unit]['State'] == 0].equals(unit_data[unit][unit_data[unit]['Cycle'] >= ts[unit]])):
        print('Not corresponding')

## Analysis of the failures

In [12]:
dev = pd.concat([aux_dev[['Unit','State']], theta_dev], axis=1)
test = pd.concat([aux_test[['Unit','State']], theta_test], axis=1)

In [13]:
unit_data = {}

for unit in list(dev['Unit'].unique()):
    
    unit_data[unit] = dev[(dev['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)
    
for unit in list(test['Unit'].unique()):
    
    unit_data[unit] = test[(test['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)

All the rotating sub-components of the engine i.e., fan, low pressure compressor (LPC), high pressure compressor (HPC), low pressure turbine (LPT) and high pressure turbine (HPT) can be affected by degradation in flow and efficiency. So there are a total of 5 components and 10 possible types of degradation. However the distinctive failure modes are only 7.
As an example Dataset 2, has two different failure modes: HPT eff and LPT flow/eff + HPT eff.

In [14]:
unit_theta = {}

for unit in list(dev['Unit'].unique()):
    
    unit_theta[unit] = unit_data[unit][unit_data[unit] != 0].dropna(how='any', axis=1).drop('Unit', axis=1)
    
    components = []
    for component in unit_theta[unit].columns:
        components.append(component[:3])
    
    print(f"The components of dev unit {unit} affected by degradation: ", end = "")
    print(*set(components), sep=", ", end=" (")
    print(*unit_theta[unit].columns, sep=", ", end = " )\n")
    
for unit in list(test['Unit'].unique()):
    
    unit_theta[unit] = unit_data[unit][unit_data[unit] != 0].dropna(how='any', axis=1).drop('Unit', axis=1)
    
    components = []
    for component in unit_theta[unit].columns:
        components.append(component[:3])

    print(f"The components of test unit {unit} affected by degradation: ", end = "")
    print(*set(components), sep=", ", end=" (")
    print(*unit_theta[unit].columns, sep=", ", end = " )\n")

The components of dev unit 1 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 2 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 3 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 4 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 5 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 6 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 7 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 8 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of dev unit 9 affected by degradation: LPT, HPT (HPT_eff_mod, LPT_eff_mod, LPT_flow_mod )
The components of test unit 10 affected by degradation:

In [15]:
normal_theta = {}
abnormal_theta = {}

for unit in list(dev['Unit'].unique()):
    
    cols = unit_theta[unit].columns
    normal_theta[unit] = unit_data[unit][unit_data[unit]['State'] == 1][cols]
    print(f'The mean for the dev unit {unit} in normal condition:\n'
          f'{normal_theta[unit].drop_duplicates().mean()}')
    abnormal_theta[unit] = unit_data[unit][unit_data[unit]['State'] == 0][cols]
    print(f'The mean for the dev unit {unit} in abnormal condition:\n'
          f'{abnormal_theta[unit].drop_duplicates().mean()}\n')

    
for unit in list(test['Unit'].unique()):

    cols = unit_theta[unit].columns
    normal_theta[unit] = unit_data[unit][unit_data[unit]['State'] == 1][cols]
    print(f'The mean for the test unit {unit} in normal condition:\n'
          f'{normal_theta[unit].drop_duplicates().mean()}')
    abnormal_theta[unit] = unit_data[unit][unit_data[unit]['State'] == 0][cols]
    print(f'The mean for the test unit {unit} in abnormal condition:\n'
          f'{abnormal_theta[unit].drop_duplicates().mean()}\n')
    

The mean for the dev unit 1 in normal condition:
HPT_eff_mod    -0.000643
LPT_eff_mod    -0.000645
LPT_flow_mod   -0.001138
dtype: float64
The mean for the dev unit 1 in abnormal condition:
HPT_eff_mod    -0.001922
LPT_eff_mod    -0.001988
LPT_flow_mod   -0.007010
dtype: float64

The mean for the dev unit 2 in normal condition:
HPT_eff_mod    -0.000650
LPT_eff_mod    -0.000503
LPT_flow_mod   -0.000311
dtype: float64
The mean for the dev unit 2 in abnormal condition:
HPT_eff_mod    -0.002728
LPT_eff_mod    -0.002701
LPT_flow_mod   -0.005757
dtype: float64

The mean for the dev unit 3 in normal condition:
HPT_eff_mod    -0.000312
LPT_eff_mod    -0.001109
LPT_flow_mod   -0.000362
dtype: float64
The mean for the dev unit 3 in abnormal condition:
HPT_eff_mod    -0.001922
LPT_eff_mod    -0.003659
LPT_flow_mod   -0.006573
dtype: float64

The mean for the dev unit 4 in normal condition:
HPT_eff_mod    -0.000739
LPT_eff_mod    -0.000496
LPT_flow_mod   -0.000733
dtype: float64
The mean for the d

The between-flight maintenance is not explicitly modeled but is considered by the process noise, so the engine health parameters (flow and efficiency) can improve within allowable limits at any point and hence the loss in efficiency or flow is not locally monotonic 

## Analysis of the observations and timestamps

In [16]:
dev = pd.concat([aux_dev, y_dev], axis=1)
test = pd.concat([aux_test, y_test], axis=1)

In [17]:
unit_data = {}

for unit in list(dev['Unit'].unique()):
    
    unit_data[unit] = dev[(dev['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)
    
for unit in list(test['Unit'].unique()):
    
    unit_data[unit] = test[(test['Unit'] == unit)]
    unit_data[unit].reset_index(drop=True, inplace=True)


In the previous version, the sampling rate was defined at 1 Hz (1 sample every second). As it can be seen the sampling rate mismatches the relative flight class, however it doesn't differ too much from it and it isn't all the same

In [18]:
# Sampling rate in Hz
hertz = 1

for unit in list(dev['Unit'].unique()):
        
    in_h = unit_data[unit]['Class'].unique()[0] * 2 - 1
    end_h = in_h + 2
    
    print(f"\nUnit {unit} has flight class: {unit_data[unit]['Class'].unique()[0]} ({in_h}-{end_h} hours)\n")
    
    max_v = 0
    min_v = len(dev)

    for cycle in  range (1, unit_data[unit]["Cycle"].iloc[-1] + 1):

        cycle_len = len(unit_data[unit][unit_data[unit]["Cycle"] == cycle])
        
        if cycle_len >= max_v:
            max_v = cycle_len
            
        if cycle_len <= min_v:
            min_v = cycle_len
        
        time = str(datetime.timedelta(seconds= cycle_len // hertz)) 
               
        if int(time[0]) >= in_h and int(time[0]) <= end_h:
            print(colored(f"Cycle {cycle} lasts {time} hours ", 'green'))
        else:
            print(colored(f"Cycle {cycle} lasts {time} hours ", 'red'))
    
    print(f'\nMin value is {min_v}, max_value is {max_v}')
        
for unit in list(test['Unit'].unique()):
        
    in_h = unit_data[unit]['Class'].unique()[0] * 2 - 1
    end_h = in_h + 2
    
    print(f"\nUnit {unit} has flight class: {unit_data[unit]['Class'].unique()[0]} ({in_h}-{end_h} hours)\n")
    
    max_v = 0
    min_v = len(dev)
    
    for cycle in  range (1, unit_data[unit]["Cycle"].iloc[-1] + 1):

        cycle_len = len(unit_data[unit][unit_data[unit]["Cycle"] == cycle])
        
        if cycle_len >= max_v:
            max_v = cycle_len
            
        if cycle_len <= min_v:
            min_v = cycle_len
        
        time = str(datetime.timedelta(seconds= cycle_len // hertz)) 
        
        if int(time[0]) >= in_h and int(time[0]) <= end_h:
            print(colored(f"Cycle {cycle} lasts {time} hours ", 'green'))
        else:
            print(colored(f"Cycle {cycle} lasts {time} hours ", 'red'))
        
    print(f'\nMin value is {min_v}, max_value is {max_v}')



Unit 1 has flight class: 1 (1-3 hours)

Cycle 1 lasts 1:34:37 hours 
Cycle 2 lasts 0:52:01 hours 
Cycle 3 lasts 1:14:22 hours 
Cycle 4 lasts 1:06:48 hours 
Cycle 5 lasts 1:34:37 hours 
Cycle 6 lasts 0:55:36 hours 
Cycle 7 lasts 1:14:58 hours 
Cycle 8 lasts 1:02:30 hours 
Cycle 9 lasts 1:25:30 hours 
Cycle 10 lasts 1:17:40 hours 
Cycle 11 lasts 1:06:48 hours 
Cycle 12 lasts 1:02:30 hours 
Cycle 13 lasts 1:09:41 hours 
Cycle 14 lasts 1:14:58 hours 
Cycle 15 lasts 1:14:58 hours 
Cycle 16 lasts 1:14:55 hours 
Cycle 17 lasts 1:02:30 hours 
Cycle 18 lasts 1:14:55 hours 
Cycle 19 lasts 0:58:24 hours 
Cycle 20 lasts 1:03:34 hours 
Cycle 21 lasts 1:29:15 hours 
Cycle 22 lasts 0:58:24 hours 
Cycle 23 lasts 1:34:37 hours 
Cycle 24 lasts 1:24:29 hours 
Cycle 25 lasts 1:09:41 hours 
Cycle 26 lasts 1:09:41 hours 
Cycle 27 lasts 1:25:30 hours 
Cycle 28 lasts 1:14:58 hours 
Cycle 29 lasts 1:03:34 hours 
Cycle 30 lasts 1:09:41 hours 
Cycle 31 lasts 1:06:48 hours 
Cycle 32 lasts 0:58:24 hours 
Cycle 33